In [1]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [2]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

producer.flush()
t1 = time.time()


print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.55 seconds


In [3]:
import pandas as pd

df_green = pd.read_csv('green_tripdata_2019-10.csv.gz')

/var/folders/0q/_0939hcn7tb0ch1qcjyxjsxw0000gn/T/ipykernel_33966/1108095895.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_green = pd.read_csv('green_tripdata_2019-10.csv.gz')


In [4]:
columns = ['lpep_pickup_datetime','lpep_dropoff_datetime','PULocationID','DOLocationID',\
                            'passenger_count','trip_distance','tip_amount']
df_green = df_green[columns]

In [5]:
t0 = time.time()

topic_name = 'green-trips'

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    #print(row_dict)
    producer.send(topic_name, value=row_dict)

    # TODO implement sending the data here

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

took 17.29 seconds


In [6]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/03/15 09:54:05 WARN Utils: Your hostname, Shumengs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.15 instead (on interface en0)
24/03/15 09:54:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/shumengshi/.ivy2/cache
The jars for the packages stored in: /Users/shumengshi/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e8d09d85-5b83-4c97-bf62-f3f7fa90d0ac;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.4 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

24/03/15 09:54:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [15]:
from pyspark.sql.functions import current_timestamp

In [18]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/03/15 09:56:09 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0q/_0939hcn7tb0ch1qcjyxjsxw0000gn/T/temporary-5b10d8bb-b657-434f-b6d8-535b917f30b6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/15 09:56:09 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 15, 9, 47, 56, 737000), timestampType=0)


In [19]:
query.stop()

In [20]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [21]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [22]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/03/15 09:56:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0q/_0939hcn7tb0ch1qcjyxjsxw0000gn/T/temporary-45f8a7eb-9d6b-4533-ac8a-5aebbb12c979. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/15 09:56:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)


In [23]:
query.stop()

In [24]:
from pyspark.sql.functions import current_timestamp
green_stream = green_stream.withColumn("timestamp", current_timestamp())

In [26]:
query = green_stream.writeStream \
    .foreachBatch(peek) \
    .start()

24/03/15 10:11:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0q/_0939hcn7tb0ch1qcjyxjsxw0000gn/T/temporary-4c14a94b-e206-4f16-93b9-262541f6437d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/15 10:11:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0, timestamp=datetime.datetime(2024, 3, 15, 10, 11, 37, 622000))


In [27]:
query.stop()

In [34]:
popular_destinations = green_stream \
    .groupBy(F.window(F.col("timestamp"), "5 minutes"), F.col("DOLocationID")) \
    .count()

popular_destinations = popular_destinations.orderBy(F.desc("count"))

In [35]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

#query.awaitTermination()

24/03/15 10:24:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0q/_0939hcn7tb0ch1qcjyxjsxw0000gn/T/temporary-951616f4-7cdb-4e14-805e-d1fc8f113a8d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/15 10:24:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|74          |17741|
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|42          |15942|
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|41          |14061|
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|75          |12840|
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|129         |11930|
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|7           |11533|
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|166         |10845|
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|236         |7913 |
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|223         |7542 |
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|238         |7318 |
|{2024-03-15 10:20:00, 2024-03-15 10:25:00}|82          |7292 |
|{2024-